In [1]:
%reload_ext ishbook
import pandas as pd
import iql
import plus
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Look up all rep account notes where the end date was modified

-- Using the **repaccountnotes** index, and a pain in the neck regex call on the *note* field to look for these two strings
* 'endDate'
* 'New:'

-- Group by the advid, username (to find out who did it, the scummy guy) the actual note text, and time it was committed

In [2]:
%%ish

newEndDates = from repaccountnotes 90d today WHERE note=~".*endDate.*New:.*" GROUP BY advid, username, note, unixtime
output = None

In [3]:
newEndDates.tail()

,advid,username,note,unixtime,
856,3857615,NaN,[russell_snow@comcast.net] Dradis job 'Securit...,1485368331,1
857,812045,NaN,[cv@civitasseniorhealthcare.com] Dradis job 'M...,1485395401,1
858,9637199,NaN,[manager@baltcoffee.com] Dradis job 'PT Food P...,1485395403,1
859,6889446,NaN,[koko.heathrow@gmail.com] Dradis job 'Personal...,1485399363,1
860,9084223,NaN,[kferguson@a-zbusinesssolutions.com] Dradis jo...,1485405443,1


## The old favorite, create a stringed list of advertiser ids from the above query

In [4]:
advids = ','.join(str(x) for x in newEndDates.advid)

## Use another IQL pull to feed into the **advertiserevent** index to get those that were created by Agency Hosted Support

In [5]:
%%ish

agencyHostedJobs = from advertiserevent 90d today WHERE advertiser_id IN ({advids}) activity_type = CREATE_AD service_rep_id = 4158 GROUP BY service_rep_username, advertiser_id, activity_type
output = None

ISH/IQL ERRORS:
  
Error evaluating 'agencyHostedJobs = from advertiserevent 90d today WHERE advertiser_id IN ({advids}) activity_type = CREATE_AD service_rep_id = 4158 GROUP BY service_rep_username, advertiser_id, activity_type': ('Connection aborted.', error(54, 'Connection reset by peer'))


## Merge the two dataframes together on the advertiser id
-  What I'm curious about is are there other teams doing the same kind of thing?
-  **to_clipboard** is used because it's way faster to get your output using that than dl-ing to CSV And trying to play around with that mess

In [34]:
agencyHostedJobs.merge(newEndDates, how = "left", left_on = "advertiser_id", right_on = "advid").to_clipboard()